# Protocol for multi-partite approach with a trusted server
In the setting where the clients/Alice can trust the server, which is the most basic setting for the multi-partite case, there is no need for any verification. The protocol works as follows:
1. The server establishes a Bell pair with every client in the network.
2. The server performs the distribution steps, resulting in correction outcomes $m_{s_{i}}$ intended for the clients.
3. For all $i$: client $i$ measures their qubit in the $X$ basis, recording measurement outcome $m_{i}$.
4. For all $i$: server sends correction outcome $m_{s_{i}}$ to client $i$.
5. For all $i$: client $i$ receives correction outcome $m_{s_{i}}$ and flips $m_{i}$ based on the necessary corrections (see notebook for GHZ distribution steps).
6. The clients either give up a portion $p$ of their results to determine $Q_{X} = \frac{1 - \langle X_{\mathcal{N}} \rangle}{2}$, or they take it as pre-determined knowledge (in this case, $p = 0$).
7. Alice encodes her message into the other $(1-p)\times L$ states. She can encode a message that is maximally $(1-h_{2}\left(Q_{X}\right))\times (1-p)\times L$ bits long. This results in her codeword $c$ of length $(1-p)\times L$.
8. Alice encodes her codeword into her measurement outcomes $\{m_{a}\}_{k}$ by flipping every bit according to the relevant bit of the codeword.
9. For all $i$: client $i$ announces its measurement outcomes $\{m_{i}^{k}\}_{k}$.
10. For all $i$: compute every bit of the (faulty) codeword as $c_{k} = \bigoplus_{i} m_{i}^{k}$.

Step 2. and Step 3. can be performed simultaneously. Moreover, step 1,2,3 are repeated $L$ times, while step 4. and 5. can either be performed in unison with step 1,2,3, or can be bunched and send all later.